In [1]:
def parse_text(text: str) -> str:
    return text.replace("**", "{b}").replace("*", "{i}").replace("\"", "\\\"")


def close_tags(text: str) -> str:
    _open = False
    open_chars = set()
    output = ""
    for char in text:
        if char == "{":
            _open = True
        elif char == "}":
            _open = False
        elif _open:
            if char not in open_chars:
                open_chars.add(char)
                output += "{" + char + "}"
            else:
                open_chars.remove(char)
                output += "{/" + char + "}"
        else:
            output += char
    return output

skip_keywords = ["!", "."]

characters = {
    "DAISY-BOT": ("daisy", "left"),
    "BATTA S. DEVILA": ("batta", "right"),
    "MOON DOG CHEF": ("dog", "center"),
}

rooms = {
    "BEDROOM": "bedroom",
    "CASTLE": "castle",
    "KITCHEN": "kitchen",
}

In [2]:
with open("Toxic Yuri Game Jam 2025_ Script.fountain", "r") as file:
    text = file.read()

In [3]:
output: str = ""
character = None
current_string = ""
characters_present = set()
character_expressions = {}
TAB = "    "

for line in text.split("\n"):
    line = line.strip()
    if not line:
        if character or current_string:
            output += TAB
            if character:
                output += character + " "
            if current_string:
                output += f"\"{close_tags(current_string.strip())}\""
            output += f"\n{TAB}\n"
        character = None
        current_string = ""
        continue
    if character:
        current_string += parse_text(line) + " "
    else:
        character_found = False
        for char_text, (name, pos) in characters.items():
            if line.startswith(char_text):
                character = name

                expression = ""
                if "(O.S.)" in line:
                    if character in characters_present:
                        output += f"{TAB}hide {name}\n"
                elif "(" in line:
                    current_expression = character_expressions.get(character)
                    expression = " " + (
                        line[line.find("(") + 1 : line.find(")")].replace("-", "").replace(",", "").lower()
                    )
                    if not current_expression or current_expression != expression:
                        character_expressions[character] = expression
                    else:
                        expression = ""

                if character not in characters_present:
                    output += f"{TAB}show {name}{expression} at {pos}\n"
                    characters_present.add(character)
                elif expression:
                    output += f"{TAB}show {name}{expression}\n"
                break
        if not character:
            if any(line.startswith(keyword) for keyword in skip_keywords):
                output += f"{TAB}# {line[1:]}\n"
                if line.startswith(".INT"):
                    for room_text, room_name in rooms.items():
                        if room_text in line:
                            output += f"{TAB}scene {room_name}\n"
                            break
                continue
            output += f"{TAB}\"{line}\"\n"
            continue

print(output)

    # INT. BATTA'S BEDROOM, EARLY EVENING
    scene bedroom
    # OPEN on the bedroom of COUNTESS BATTA S. DEVILA, a Villaness who lives in a comically- spooky castle on the edge of the kingdom. Despite the keep's looming exterior, what we can see of Batta's room is a bit of a hodgepodge of different styles and interests- from the billowing, dramatic Victorian curtains to an evil Lightning McQueen racecar bed in the center of the room.
    # This evening, Batta awakens from her evil slumber and begins preparing for the night ahead with the help of her trusty homemade vampiric thrall, DAISY-BOT.
    show batta relaxed happy at right
    batta "Oh DAISY-BOT, my precious little iron pill, you would not {i}believe{/i} the dream I just had-"
    
    show daisy neutral at left
    daisy "My Lady, surely this might not be another one of your {i}fantasy dreams?{/i} Because I feel as your Chief of Staff, it is my duty to remind you that the floors in the West Hall corridor are {i}still{/i} not